In [ ]:
import sympy as sm
import sympy.physics.mechanics as me
import time
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, minimize
import matplotlib.pyplot as plt
import matplotlib as mp
from matplotlib import patches
%matplotlib inline
from IPython.display import HTML
mp.rcParams['animation.embed_limit'] = 2**128
from matplotlib import animation

Needed to only plot a few *hysteresis curves*

In [ ]:
class Rausspringen(Exception):
    pass

An elastic homogenious ball of radius *r* and mass *m* is thrown (or dropped) on an uneven street.\
An observer (a particle) of mass $m_o$ may be put anywhere inside the ball.

**Variables**

- $N$: inertial frame
- $A$: frame fixed to the ball

- $Dmc$: center of the ball, a *RigidBody*
- $P_o$: the observer, a *Particle*
- $P_{st}$: point where the ball hits the street.

- $q_1, q_2, q_3$: angle of the ball w.r.t. N
- $u_1, u_2, u_3$: angular velocities of the ball
- $x_b, y_b, z_b$: location of the center of the ball, relative to the inertial frame N
- $ux_b, uy_b, uz_b$: the speeds


- $c_{tau}$: the experimental constant needed for Hunt-Crossley
- $m_u$: the friction constant
- $ny_b, ny_s$: Poisson's ratio of ball and street respectively.
- $E_b, E_s$: Young's modulus of ball and street respectively
- $rhodt_{street}$: speed right before the impact.


- $x_{st}, z_{st}$: impact point, where the ball hits the street.

- $m, m_o, i_{XX}, i_{YY}, i_{ZZ}$: mass of ball, mass of observer, moments of inertia of the ball
- $amplitude, frequenz$: parameters to describe the bumpy road
- $\alpha, \beta, \gamma$: describe the location of the observer relative to the center of the ball 

NOTE: This seems very 'tricky' as far as the numerical integration is concerned. I assume, because the systen is extremely stiff during collisions.

In [ ]:
start = time.time()

N, A = sm.symbols('N, A', cls = me.ReferenceFrame)
P0, Dmc, Po, Pst = sm.symbols('P0, Dmc, Po, Pst', cls = me.Point)

q1, q2, q3, u1, u2, u3 = me.dynamicsymbols('q1, q2, q3, u1, u2, u3')
xst, zst = sm.symbols('xst, zst')
xb, yb, zb, uxb, uyb, uzb = me.dynamicsymbols('xb, yb, zb, uxb, uyb, uzb')

m, mo, g, r, k, amplitude, frequenz, alpha, beta, gamma, iXX, iYY, iZZ, t = sm.symbols('m, mo, g, r,' 
        'k, amplitude, frequenz, alpha, beta, gamma, iXX, iYY, iZZ, t')

nyb, nys, Eb, Es, rhodtstreet, ctau, mu = sm.symbols('nyb, nys, Eb, Es, rhodtstreet, ctau, mu')

A.orient_body_fixed(N, (q1, q2, q3), '123')
rot = A.ang_vel_in(N)
A.set_ang_vel(N, u1*N.x + u2*N.y + u3*N.z)
rot1 = A.ang_vel_in(N)

P0.set_vel(N, 0.)

Dmc.set_pos(P0, xb*N.x + yb*N.y + zb*N.z)
Dmc.set_vel(N, uxb*N.x + uyb*N.y + uzb*N.z)

Po.set_pos(Dmc, r * (alpha*A.x + beta*A.y + gamma*A.z))
Po.v2pt_theory(Dmc, N, A);

*Here the street is modeled*.\
Basically it is a parabola open to the top (that is the positive Y direction), with superimposed sine waves, of ever larger frequency, but smaller amplitude.\
The integer *rumpel*, larger than zero, determines how many sine waves will be used.\
At the bottom, I define $P_{st}$, the point of impact of the ball with the street. How I find it is described further down.

In [ ]:
#============================================
rumpel = 3                     
#============================================
if isinstance(rumpel, int) == False or rumpel < 1:
    raise Exception('rumpel must be an integer larger than zero')
    
def gesamt(x, z, amplitude, frequenz):
    strasse = sum([amplitude/j * (sm.sin(j*frequenz * x) + 2.5 * sm.sin(j*frequenz * z)) for j in range(1, rumpel)])
    strassen_form = (frequenz/2. * x)**2  + (frequenz/2. * z)**2
    return strassen_form + strasse 

# needed for the lambdification, as a function cannot be lambdified - at least I do not know how.
gesamt1 = gesamt(xb, zb, amplitude, frequenz) 

Pst.set_pos(P0, xst*N.x + gesamt(xst, zst, amplitude, frequenz)*N.y + zst*N.z)

Find the minimal osculating circle (Schmiegekreis) of the street. I found the formula for the one dimensional case in the internet.\
I simply take the smaller ones of the X - direction, Z - direction, respectively.\
There may the a formula for this 3D case, I did not bother to find it.

In [ ]:
r_max_xst = (sm.S(1.) + (gesamt(xst, zst, amplitude, frequenz).diff(xst))**2
           )**sm.S(3/2)/gesamt(xst, zst, amplitude, frequenz).diff(xst, 2)
r_max_zst = (sm.S(1.) + (gesamt(xst, zst, amplitude, frequenz).diff(zst))**2 
           )**sm.S(3/2)/gesamt(xst, zst, amplitude, frequenz).diff(zst, 2)

**Point of impact**\
Here I get the $x_{st}, z_{st}$ coordinates of the point where the balls hits the street.\
The idea is this:\
$vector_1$ is normal to the street at the point $P_{st}$ = ($x_{st}$, gesamt($x_{st}, z_{st}$, *amplitude, frequenz*), $z_{st}$). It points 'inwards'. I got this formula from the internet.\
$vector_2$ is the vector pointing from $P_{st}$ to the center of the ball, *Dmc*\
These two vectors must be parallel at all times, that is $\hat{vector_1}$ = $\hat{vector_2}$, with $\hat a$ meaning $\dfrac{a}{|a|}$. This means, that $P_{st}$ would be the contact point of the ball with the street, if its radius *r* was equal to |$vector_2|$.\
This gives two equations for $x_{st}, z_{st}$ to be solved numerically later, to get the correct starting values of $x_{st}, z_{st}$, and then again in every step of the numerical integration.\
This idea works at least as long as there is only one solution to the equation.

In [ ]:
vektor1 = -(gesamt(xst, zst, amplitude, frequenz).diff(xst)*N.x - N.y + 
           gesamt(xst, zst, amplitude, frequenz).diff(zst)*N.z).normalize()

vektor2 = Dmc.pos_from(Pst).normalize()

auftreff = vektor1 - vektor2

auftreffX = me.dot(auftreff, N.x)
auftreffZ = me.dot(auftreff, N.z)

loesung = sm.Matrix([auftreffX, auftreffZ])
print('loesung DS', me.find_dynamicsymbols(loesung))
print('loesung free symbols', loesung.free_symbols)
print('loesung has {} operations'.format(sum([loesung[i].count_ops(visual=False) for i in range(2)])))

**Function which calculates the forces of an impact between the ball and the street**\
1.\
The impact force is on the line normal to the street, going through the center of the ball, $P_1$. I use Hunt Crossley's method to calculate it\
It's direction points from the street to the ball.\
2.\
The friction force acting on the contact point $CP_0$ is proportional to the component of the speed of $CP_0$ in the plane tanget to the street, the impact force and a friction factor $m_{uW}$.\
It is equivalent to a force through $P_1$ and to a torque acting on $A_1$, the ball fixed frame.


**Note about the force during the collisions**

**Hunt Crossley's method**
 
My reference is this article, given to me by JM\
https://www.sciencedirect.com/science/article/pii/S0094114X23000782 \

 
This is with dissipation during the collision, the general force is given in (63) as\
$f_n = k_0 \cdot \rho + \chi \cdot \dot \rho$, with $k_0$ as above, $\rho$ the penetration, and $\dot\rho$ the speed of the penetration.\
In the article it is stated, that $n = \frac{3}{2}$ is a good choice, it is derived in Hertz' approach. Of course, $\rho, \dot\rho$ must be the signed magnitudes of the respective vectors.

A more realistic force is given in (64) as:\
$f_n = k_0 \cdot \rho^n + \chi \cdot \rho^n\cdot \dot \rho$, as this avoids discontinuity at the moment of impact.

**Hunt and Crossley** give this value for $\chi$, see table 1:

$\chi = \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{k_0}{\dot \rho^{(-)}}$, 
where $c_\tau = \dfrac{v_1^{(+)} - v_2^{(+)}}{v_1^{(-)} - v_2^{(-)}}$, where $v_i^{(-)}, v_i^{(+)}$ are the speeds of $body_i$, before and after the collosion, see (45), $\dot\rho^{(-)}$ is the speed right at the time the impact starts. $c_\tau$ is an experimental factor, apparently around 0.8 for steel.

Using (64), this results in their expression for the force:

$f_n = k_0 \cdot \rho^n \left[1 + \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{\dot\rho}{\dot\rho^{(-)}}\right]$

with $k_0 = \frac{4}{3\cdot(\sigma_1 + \sigma_2)} \cdot \sqrt{\frac{R_1 \cdot R_2}{R_1 + R_2}}$, where $\sigma_i = \frac{1 - \nu_i^2}{E_i}$, with $\nu_i$ = Poisson's ratio, $E_i$ = Young"s modulus, $R_1, R_2$ the radii of the colliding bodies, $\rho$ the penetration depth. All is near equations (54) and (61) of this article.\
From the ball's perspective, the street is concave. In the formula I simply take $R_2 < 0$. As the street has different osculating radii, I take the average of the radii in the X / Z directions. As |osculating radius| > |radius of the ball|, the term of the square root should always be positive.\
I am not sure, whether this applies the formula correctly.

As per the article, $n = \frac{3}{2}$ is always to be used.

*spring energy* =   $ k_0 \cdot \int_{0}^{\rho} k^{3/2}\,dk$ = $k_0 \cdot\frac{2}{5} \cdot \rho^{5/2}$\
I assume, the dissipated energy cannot be given in closed form, at least the article does not give one.

*Note*  
$c_\tau = 1.$ gives **Hertz's** solution to the impact problem, also described in the article.


**Friction when the ball hits the street**

This website:
https://math.stackexchange.com/questions/2195047/solve-the-vector-cross-product-equation

gives: $b = a \times x \rightarrow x = \dfrac{b \times a}{|a|^2}$

This way, I can easily get the force acting on CP0 ( = contact point of ball with wall), without any further geometric considerations. Of course the friction force has opposite to the speed of CP0\
The friction force on CP0 is equivalent to a force on P1 (center of the ball) and a torque on A1 (ball fixed frame)

In the formula for the impact force, *forcec* I use |$rhodtstreet$| and  **-**$rhodt$. The reason is this:
- As the diagram below shows, the speed jumps dramatically during impact. Hence when I try to get $\dot \rho^{(-)}$ it may be positive or negative.
- As per my definition the speed component in the first phase of the impact is negative. Hence during the first phase of the impact, $\dfrac{\dot \rho}{\dot \rho^{(-)}} > 0.$, as it should be.

*vorzeichen* is introduced to ensure, that the collision force continues to act if the ball falls through the street.

In [ ]:
def HC_street(N, A1, P1, r, ctau, rhodtstreet):
        
    vektor = P1.pos_from(Pst)
    vorzeichen = sm.sign(me.dot(vektor, N.y)) # positive if the ball is above the street, lse negative.
    abstand = vektor.magnitude()
    richtung = vektor.normalize()
    rho = r - abstand       # positive in the ball has penetrated the wall
    CP0 = me.Point('CP0')
    CP0.set_pos(P1, -vektor)
    vCP0 = CP0.v2pt_theory(Dmc, N, A1)
    rhodt = me.dot(vCP0, richtung)             #me.dot(P1.pos_from(P0).diff(t, N), richtung)
    rho = sm.Abs(rho) # sm.Max(rho, sm.S(0))

#determine R2 of the a.m. formula    
    r_xst   = -(sm.S(1.) + (gesamt(xst, zst, amplitude, frequenz).diff(xst))**2
               )**sm.S(3/2)/gesamt(xst, zst, amplitude, frequenz).diff(xst, 2)
    r_zst   = -(sm.S(1.) + (gesamt(xst, zst, amplitude, frequenz).diff(zst))**2 
               )**sm.S(3/2)/gesamt(xst, zst, amplitude, frequenz).diff(zst, 2)
    R2      = 0.5 * (r_xst + r_zst)
# determine k0
# the sm.Abs(..) should theoretically not be necessary. But seems to avoid issues at times.
    wurzel = sm.sqrt(sm.Abs(r * R2 /(r + R2)))
    sigma_b = (1. - nyb**2) / Eb
    sigma_s = (1. - nys**2) / Es
    k0      = 4. / (3.*(sigma_b + sigma_s)) * wurzel

# Here I assume, that the penetration is small, otherwise I would not know how to do it.
    forcec = k0 * rho**(3/2) * ( 1. + 3./2. * (1 - ctau) * (-rhodt) / sm.Abs(rhodtstreet)) * (richtung *
            sm.Heaviside(r - vorzeichen * abstand, 0.))
    
    vx = vCP0 - (me.dot(vCP0, richtung)) * richtung
    friction_force = forcec.magnitude() * mu * (-vx)        
    hilfs = CP0.pos_from(P1)
    torque = hilfs.cross(friction_force) * sm.Heaviside(r - abstand, 0.)
    forcef = 1./me.dot(hilfs, hilfs) * torque.cross(hilfs) * sm.Heaviside(r - abstand, 0.)
        
    return [forcec, forcef, torque]

**Kane's equations**. 

In [ ]:
start2 = time.time()

I = me.inertia(A, iXX, iYY, iZZ)
body = me.RigidBody('body', Dmc, A, m, (I, Dmc))
Poa  = me.Particle('Poa', Po, mo)
BODY = [body, Poa]

F1 = [(Dmc, -m*g*N.y), (Po, -mo*g*N.y)]
F2 = [(Dmc, HC_street(N, A, Dmc, r, ctau, rhodtstreet)[0] + HC_street(N, A, Dmc, r, ctau, rhodtstreet)[1])]
F3 = [(A, HC_street(N, A, Dmc, r, ctau, rhodtstreet)[2])]

FL = F1 + F2 + F3

q_ind = [xb, yb, zb] + [q1, q2, q3]
u_ind = [uxb, uyb, uzb] + [u1, u2, u3]

kd = [me.dot(rot - rot1, uv) for uv in A] + [uxb - xb.diff(t), uyb - yb.diff(t), uzb - zb.diff(t)]

KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM = KM.mass_matrix_full
force = KM.forcing_full

print('force DS', me.find_dynamicsymbols(force))
print('force free symbols', force.free_symbols)
print('force has {} operations'.format(sum([force[i].count_ops(visual=False) 
                for i in range(len(force))])), '\n')

print('MM DS', me.find_dynamicsymbols(MM))
print('MM free symbols', MM.free_symbols)
print('MM has {} operations'.format(sum([MM[i, j].count_ops(visual=False) 
                for i in range(MM.shape[0]) for j in range(MM.shape[1])])), '\n')

print('It took {:.3f} sec to establish Kanes equations, for rumpel = {}'.format(time.time() - start2
        , rumpel))

Here some *functions*, needed further down are defined:
 - $abstand_1$: needed to calculate the distance of the center of the ball from the street
 - $rhodtstreet_1$: needed to calculate the speed of the impact.
 
 
I like to look at the *energies of the system*. This has often (just about always!) told me, I had made a mistake in the equations of motion.\
So, these functions are defined here.

In [ ]:
pot_energie = m * g * me.dot(Dmc.pos_from(P0), N.y) + mo * g * me.dot(Po.pos_from(P0), N.y)
kin_energie = sum([koerper.kinetic_energy(N) for koerper in BODY])

rho  = Dmc.pos_from(Pst).magnitude()
rho1 = sm.Max(r - rho, sm.S(0)) # penetration depth, positive if there is penetration.

#determine R2 of the a.m. formula    
r_xst   = -(sm.S(1.) + (gesamt(xst, zst, amplitude, frequenz).diff(xst))**2
               )**sm.S(3/2)/gesamt(xst, zst, amplitude, frequenz).diff(xst, 2)
r_zst   = -(sm.S(1.) + (gesamt(xst, zst, amplitude, frequenz).diff(zst))**2 
               )**sm.S(3/2)/gesamt(xst, zst, amplitude, frequenz).diff(zst, 2)
R2      = 0.5 * (r_xst + r_zst)

# determine k0    
wurzel = sm.sqrt(sm.Abs(r * R2 /(r + R2))) # theoretically the sm.Abs(..) should not be needed.
sigma_b = (1. - nyb**2) / Eb
sigma_s = (1. - nys**2) / Es
k0      = 4. / (3.*(sigma_b + sigma_s)) * wurzel
spring_energie = 2./5. * k0 * sm.Heaviside(r - rho, 0.) * rho1**(5/2)

vektor = Dmc.pos_from(Pst)
abstand1 = vektor.magnitude()

richtung = vektor.normalize()
CP0 = me.Point('CP0')
CP0.set_pos(Dmc, -vektor)
vCP0 = CP0.v2pt_theory(Dmc, N, A)
rhodtstreet1 = me.dot(vCP0, richtung)
print('vCP0 DS', me.find_dynamicsymbols(vCP0, reference_frame=N))
print('vCP0 FS', vCP0.free_symbols(reference_frame=N))

Convert the *sympy functions* to *numpy functions* so numerical evaluations may be done. 

In [ ]:
qL   = q_ind + u_ind
pL   = [xst, zst] + [m, mo, g, r, amplitude, frequenz, alpha, beta, gamma, iXX, iYY, iZZ
      ] + [nyb, nys, Eb, Es, ctau, mu] + [rhodtstreet]
pL1  = [xst, zst] + [m, mo, g, r, amplitude, frequenz, alpha, beta, gamma, iXX, iYY, iZZ
      ] + [nyb, nys, Eb, Es, ctau, mu]
pL11 = [m, mo, g, r, amplitude, frequenz, alpha, beta, gamma, iXX, iYY, iZZ
      ] + [nyb, nys, Eb, Es, ctau, mu]
    
MM_lam = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True)

gesamt_lam = sm.lambdify([xb, zb, amplitude, frequenz], gesamt1, cse=True)

loesung_lam = sm.lambdify([xst, zst] + [xb, yb, zb] + [amplitude, frequenz], loesung, cse=True)

r_max_lam = sm.lambdify([xst, zst] + pL11, [r_max_xst, r_max_zst], cse=True)

pot_lam    = sm.lambdify(qL + pL, pot_energie, cse=True)
kin_lam    = sm.lambdify(qL + pL, kin_energie, cse=True)
spring_lam = sm.lambdify(qL + pL, spring_energie, cse=True)

abstand1_lam = sm.lambdify(qL + pL, abstand1, cse=True)

rhodtstreet1_lam = sm.lambdify(qL + pL1, rhodtstreet1, cse=True)

The *parameters* and the *initial conditions* are set here. For their meaning, see above.\
It makes sense to use names close to the names used to set up Kane's equations, but they should **not** be the same ones. The sm.symbols and the me.dynamic symbols would be overwritten - with unintended consequences.

For reasons I do not understand, the *no energy loss* case, that is $c_{tau} \approx 1.$ and $m_u \approx 0.$ gives numerical problems. 

In [ ]:
#==============================================================
# Set parameters and initial conditions.
xb1, yb1, zb1 = 0., 15., 0.
uxb1, uyb1, uzb1 = -1., 1., 1.

q11, q21, q31 = 0., 0., 0.
u11, u21, u31 = 0., 0., 0.

m1 = 1.
mo1 = 0.1
r1 = 1.

nyb1  = 0.5   # Poisson"s ratio for rubber, from the internet
nys1  = 0.15  # Poisson's ration for concrete, from the internet
Eb1   = 3.e8  # Young's modulus for rubber is really about 3.e9
Es1   = 2.e10 # Young's modulus for steel is really about 2.e11

ctau1 = 0.85    # experimental constant
mu1   = 0.1    # coefficient of friction

alpha1, beta1, gamma1 = 0.7, 0., 0.7
amplitude1 = 1.
frequenz1 = .15

intervall = 7.5

#===============================================================
if alpha1**2 + beta1**2 + gamma1**2 >= 1.:
    raise Exception('Observer is outside of the ball')
    
schritte     = int(intervall * 20000.) # this should be slightly less than nfev

iXX1         = 2./5. * m1 * r1**2      # from the internet.
iYY1         = iXX1
iZZ1         = iXX1
rhodtstreet1 = 1. # unimportant value, will be overwritten before used.

pL_vals1  = [m1, mo1, 9.81, r1, amplitude1, frequenz1, alpha1, beta1, gamma1, iXX1, iYY1, iZZ1
           ] + [nyb1, nys1, Eb1, Es1, ctau1, mu1] + [rhodtstreet1]
pL1_vals1 = [m1, mo1, 9.81, r1, amplitude1, frequenz1, alpha1, beta1, gamma1, iXX1, iYY1, iZZ1
           ] + [nyb1, nys1, Eb1, Es1, ctau1, mu1]

The ball must contact the street at *one* point only. Here I check, that the radius of the ball, $r_1$ is smaller than the smallest osculation circle (Schmiegekreis) of the street.\
if the radius of the ball is close to the osculating radius, things 'break down'. I suppose numerical issues.

In [ ]:
if frequenz1 > 0.:
    def func2(x, args):
# just needed to get the arguments matching for minimuze
        return np.abs(r_max_lam(*x, *args)[0])

    def func3(x, args):
# just needed to get the arguments matching for minimuze
        return np.abs(r_max_lam(*x, *args)[1])

    x0 = (0.1, 0.1)      # initial guess
    minimal1 = minimize(func2, x0, pL1_vals1)
    minimal2 = minimize(func3, x0, pL1_vals1)

    minimal = min(minimal1.get('fun'), minimal2.get('fun'))

    if pL_vals1[3] < minimal:
        print('selected radius = {} is less than minimally admissible radius = {:.4f}, hence o.k.'
          .format(pL_vals1[3], minimal), '\n')
    else:
        print('selected radius {} is larger than admissible radius {:.4f}, hence NOT o.k.'
          .format(pL_vals1[3], minimal), '\n')
        raise Exception('Radius of wheel is too large')

else:
    print('the street is a horizontal plane')

The initial values of $x_{st}, z_{st}$ are found numerically. I iterate twice, to get improved results, sometimes.\
If the ball is not above the street, an exception is raised.

In [ ]:
def func1(x0, args):
    return loesung_lam(*x0, *args).reshape(2)

x0 = [1., 1.]                                 # initial guess
args = [xb1, yb1, zb1, amplitude1, frequenz1]
for _ in range(2):
    antwort = fsolve(func1, x0, args)
    x0 = antwort                              # improved guess

antwort = [antwort[0], antwort[1]]

min_hoehe = gesamt_lam(xb1, zb1 , amplitude1, frequenz1)
if yb1 > min_hoehe:
    print('minimal height of the ball above P0 must be {:.3f} < yb1 = {:.3f}, so this is fine.'
      .format(min_hoehe, yb1))
else:
    raise Exception('yb1 too small, must be at least {:.3f}'.format(min_hoehe))

**Numerical Integration**

*Comments*
- max_step must be small in *solve_ivp*, else the integration may miss the points of the collision of the ball with the street.
- The values of the collision coordinate, $x_{st}, z_{st}$ are available only during integration. They are collected in *zusatz1*.
- I iterate over the numerical solution for $x_{st}, z_{st}$ a couple of times, this sometimes may improve the accuracy of the solution.

In [ ]:
start2 = time.time()
max_step = 0.0001
pL_vals  = antwort + pL_vals1
pL1_vals = antwort + pL1_vals1
print('starting parameters are', '\n', ['{:.3f}'.format(pL_vals[i]) for i in range(len(pL_vals))], '\n')

times = np.linspace(0, intervall, schritte)
t_span = (0., intervall)

y0 = [xb1, yb1, zb1, q11, q21, q31] + [uxb1, uyb1, uzb1, u11, u21, u31]
print('initial values are', '\n', ['{:.3f}'.format(y0[i]) for i in range(len(y0))], '\n')

zusatz1 = []
x0 = antwort

def gradient(t, y, args):
    global x0
    args1 = [y[0], y[1], y[2], args[6], args[7]]
# Iteration, to, hopefully, improve the accuracy of the solution
    for _ in range(2):
        antwort = fsolve(func1, x0, args1)
        x0 = antwort
    args[0] = antwort[0]
    args[1] = antwort[1]
    zusatz1.append([t, args[0], args[1]])
    
    if max_step < abstand1_lam(*y, *args) - r1 < 2. * max_step:
        args[-1] = rhodtstreet1_lam(*y, *pL1_vals)
    sol = np.linalg.solve(MM_lam(*y, *args), force_lam(*y, *args))
    return np.array(sol).T[0]
        
resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), method='BDF', max_step=max_step
            , atol=1.e-9, rtol=1.e-9)

resultat = resultat1.y.T
print('resultat shape', resultat.shape, '\n')
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ', message is:', resultat1.message, '\n')

print("To numerically integrate an intervall of {} sec the routine cycled {} times and it took {:.3f} sec "
      .format(intervall, resultat1.nfev, time.time() - start2), '\n')

Plot any **generalized coordinates or speeds** you want to see.\
*times* contains many points in time, needed to get the energies right, and also the hysteresis curves. Here this is not needed, so I reduce the points considered to around *zeitpunkte*.

In [ ]:
# reduce the number of points of time to around zeitpunkte
times2 = []
resultat2 = []
index2 = []

#=======================
zeitpunkte = 500
#=======================

reduction = max(1, int(len(times)/zeitpunkte))

for i in range(len(times)):
    if i % reduction == 0:
        times2.append(times[i])
        resultat2. append(resultat[i])
schritte2 = len(times2)
resultat2 = np.array(resultat2)
times2 = np.array(times2)


bezeichnung = ['xb1', 'yb1', 'zb1', 'q11', 'q21', 'q31'] + ['uxb1', 'uyb1', 'uzb1', 'u11', 'u21', 'u31']
fig, ax = plt.subplots(figsize=(10,5))
for i in (0, 1, 2, 9, 10, 11):
    ax.plot(times2, resultat2[:, i], label=bezeichnung[i])
ax.set_xlabel('time (sec)')
ax.set_title('Generalized coordinates and / or speeds ')
ax.legend();

The locations of the contact point, $x_{st}$, $z_{st}$ are available only during the integration, where I collect them in *zusatz1*, see the function *gradient* above.\
Here I match them as closely as I can to the points in time stored in *times*. *index* will store the locations, where the match is close.\
This takes a may take a long time to run, as there are essentially $\frac{1}{2} \cdot |times|^2$ operations. 

In [ ]:
zusatz1 = np.array(zusatz1)
zaehler = 0
index = []

for zeit in times:
    zaehler1 = np.min(np.argwhere(np.array(zusatz1[zaehler :, 0] >= zeit)))
    index.append(zaehler+zaehler1)
    zaehler = zaehler1
if len(index) != len(times[: resultat.shape[0]]):
    raise Exception('Something happened')

Plot the **energies** of the system.

In [ ]:
pot_np    = np.empty(resultat.shape[0])
kin_np    = np.empty(resultat.shape[0])
spring_np = np.empty(resultat.shape[0])
total_np  = np.empty(resultat.shape[0])

for i in range(resultat.shape[0]):
    pL_vals[0]   = zusatz1[index[i], 1]
    pL_vals[1]   = zusatz1[index[i], 2]
    pot_np[i]    = pot_lam(*[resultat[i, j] for j in range(12)], *pL_vals)
    kin_np[i]    = kin_lam(*[resultat[i, j] for j in range(12)], *pL_vals)
    spring_np[i] = spring_lam(*[resultat[i, j] for j in range(12)], *pL_vals)
    total_np[i]  = pot_np[i] + kin_np[i] + spring_np[i]

if ctau1 == 1. and mu1 == 0.:
    max_total, min_total = np.max(total_np), np.min(total_np)
    print('Max. deviation of the total energy from being constant is {:.2e} % of max. total energy'.
          format((max_total - min_total)/max_total * 100))
    
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(times[: resultat.shape[0]], pot_np, label='potential energy')
ax.plot(times[: resultat.shape[0]], kin_np, label='kinetic energy')
ax.plot(times[: resultat.shape[0]], spring_np, label='spring energy')
ax.plot(times[: resultat.shape[0]], total_np, label='total energy')
ax.set_xlabel('time (sec)')
ax.set_title('Energies of the system')
ax.legend();

Here I plot the speed component of the ball in the $\overline{Pst \to Dmc}$ direction.\
Obviously it jumps almost instantaneoulsly during impact.

In [ ]:
test_np = np.empty(resultat.shape[0])
for i in range(resultat.shape[0]):
    pL1_vals[0]   = zusatz1[index[i], 1]
    pL1_vals[1]   = zusatz1[index[i], 2]
    test_np[i] = rhodtstreet1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL1_vals)
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(times[: resultat.shape[0]], test_np)
ax.set_xlabel('time (sec)')
ax.set_title('speed component in impact direction');

Plot the **hysteresis curves** of successive impacts of the ball with the street.\
The red numbers are the times, approximately, when the impacts took place.\
As there may be many impacts, the picture may look messy. With the parameter *hyst* one may set the number of curves to be plotted.

In [ ]:
#==========================
hyst = 2
#==========================

#determine R2 of the a.m. formula    
r_xst   = -(sm.S(1.) + (gesamt(xst, zst, amplitude, frequenz).diff(xst))**2
               )**sm.S(3/2)/gesamt(xst, zst, amplitude, frequenz).diff(xst, 2)
r_zst   = -(sm.S(1.) + (gesamt(xst, zst, amplitude, frequenz).diff(zst))**2 
               )**sm.S(3/2)/gesamt(xst, zst, amplitude, frequenz).diff(zst, 2)

r_xst_lam = sm.lambdify([xst, zst, amplitude, frequenz], r_xst, cse=True)
r_zst_lam = sm.lambdify([xst, zst, amplitude, frequenz], r_zst, cse=True)

   
sigma_b = (1. - nyb1**2) / Eb1
sigma_s = (1. - nys1**2) / Es1

HC_kraft = []
HC_displ = []
HC_times = []
zaehler  = 0
zaehler1 = 0
i0 = 0
    
for i in range(resultat.shape[0]):
    pL_vals[0]    = zusatz1[index[i], 1]
    pL_vals[1]    = zusatz1[index[i], 2]
    pL1_vals[0]   = zusatz1[index[i], 1]
    pL1_vals[1]   = zusatz1[index[i], 2]
    
    xst1 = r_xst_lam(pL_vals[0], pL_vals[1], amplitude1, frequenz1)
    zst1 = r_zst_lam(pL_vals[0], pL_vals[1], amplitude1, frequenz1)
    R2   = 0.5 * (xst1 + zst1)
    wurzel = sm.sqrt(sm.Abs(r1 * R2 /(r1 + R2))) # theoretically no need for sm.Abs(...)
    k0      = 4. / (3.*(sigma_b + sigma_s)) * wurzel
    
    abstand = r1  - abstand1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
            *pL_vals)
    if abstand < 0.:
        i0 = i+1

    if abstand >= 0. and i0 == i:
        zaehler1 += 1
        walldt = rhodtstreet1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL1_vals)
    try:
        if zaehler1 > hyst:
            raise Rausspringen
        if abstand >= 0.:
            rhodt = rhodtstreet1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL1_vals)
            kraft0 = k0 * abstand**(3/2) * (1. + 3./2. * (1 - ctau1) * rhodt/walldt)
            HC_displ.append(abstand)
            HC_kraft.append(kraft0)
            HC_times.append((zaehler, times[i]))
            zaehler +=1
    except Rausspringen:
        break
    
HC_displ = np.array(HC_displ)
HC_kraft = np.array(HC_kraft)

# print only, if there were collisions with the street.    
if len(HC_displ) != 0:
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(HC_displ, HC_kraft, color='green')
    ax.set_xlabel('penetration depth (m)')
    ax.set_ylabel('contact force (N)')
    ax.set_title(f'hysteresis curves of successive impacts of the ball_1 with the street, ctau = {ctau1}, mu = {mu1}')
    
    zeitpunkte = 8
    reduction = max(1, int(len(HC_times)/zeitpunkte))
    for k in range(len(HC_times)):
        if k % reduction == 0:
            coord  = HC_times[k][0]
            ax.text(HC_displ[coord], HC_kraft[coord], f'{HC_times[k][1]:.4f}', color="red")

*Animate the movement*\
The color of the ball indicates its height above $P_0$\
*times* may have many entries, which makes the animation, at least if *HTML(..)* has to be used, take a long time to calculate. I reduce the number of entries to around *zeitpunkte*, maybe at the risk of loosing some accuracy.

In [ ]:
CP_pos_lam = sm.lambdify(qL + pL, [me.dot(Dmc.pos_from(P0), uv) for uv in N], cse=True)
Po_pos_lam = sm.lambdify(qL + pL, [me.dot(Po.pos_from(P0), uv) for uv in (N.x, N.z)], cse=True)

# reduce the number of points of time to around zeitpunkte
times2 = []
resultat2 = []
index2 = []

#=======================
zeitpunkte = 500
#=======================

reduction = max(1, int(len(times)/zeitpunkte))

for i in range(len(times)):
    if i % reduction == 0:
        times2.append(times[i])
        resultat2. append(resultat[i])
        index2.append(index[i]) 
schritte2 = len(times2)
resultat2 = np.array(resultat2)
times2 = np.array(times2)

CPx  = np.empty(schritte2)
CPy  = np.empty(schritte2)
CPy1 = np.empty(schritte2)
CPz  = np.empty(schritte2)

Pox = np.empty(schritte2)
Poz = np.empty(schritte2)

for l in range(schritte2):
    pL_vals[0]   = zusatz1[index2[l], 1]
    pL_vals[1]   = zusatz1[index2[l], 2]
    CPx[l] = CP_pos_lam(*[resultat2[l, j] for j in range(resultat2.shape[1])], *pL_vals)[0]
    CPy[l] = CP_pos_lam(*[resultat2[l, j] for j in range(resultat2.shape[1])], *pL_vals)[1]
    CPz[l] = CP_pos_lam(*[resultat2[l, j] for j in range(resultat2.shape[1])], *pL_vals)[2]
    
    Pox[l] = Po_pos_lam(*[resultat2[l, j] for j in range(resultat2.shape[1])], *pL_vals)[0]
    Poz[l] = Po_pos_lam(*[resultat2[l, j] for j in range(resultat2.shape[1])], *pL_vals)[1]
    

# needed to give the picture the right size.
xmin1 = min(CPx)
xmin2 = min(CPz)
xmin = min(xmin1, xmin2)

xmax1 = max(CPx)
xmax2 = max(CPz)
xmax = max(xmax1, xmax2)

for i in range(len(CPy)):
    CPy1[i] = int(CPy[i] * 100.)
ymin = min(CPy1)
ymax = max(CPy1)

# This is to asign colors of 'plasma' to the points.
Test = mp.colors.Normalize(ymin, ymax)
Farbe = mp.cm.ScalarMappable(Test, cmap='plasma')
farbe1 = Farbe.to_rgba(CPy1[0]*100)    # color of the starting position
    
def animate_pendulum(zeit, x1, y1, z1, Pox1, Poz1):
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'aspect': 'equal'})
    fig.colorbar(Farbe, label='Height of the ball above ground level \n Factor = 100, the real height are the numbers divided by factor', shrink=0.9, ax=ax)
    ax.axis('on')
    ax.set_xlim(xmin - 2., xmax + 2.)
    ax.set_ylim(xmin - 2., xmax + 2.)
    ax.set_xlabel(' X Axis', fontsize=18)
    ax.set_ylabel('Z Axis', fontsize=18)

    line1, = ax.plot([], [], 'o', color=farbe1, markersize=35)                     # ball
    line2, = ax.plot([], [], color='blue', linewidth=0.25)           # to trace the movement of CP
    line3, = ax.plot([], [], 'o', color='red', markersize=5)          # the observer
    
    def animate(i):
        farbe2 = Farbe.to_rgba(y1[i]*100)                                # color of the actual point at time i
        ax.set_title('running time {:.2f} sec, with rumpel = {}'.format(zeit[i], rumpel), fontsize=15)
        line1.set_data([x1[i]], [z1[i]])
        line1.set_color(farbe2)
        line2.set_data(x1[: i], z1[: i])  
        line3.set_data([Pox1[i]], [Poz1[i]])
        
        return line1, line2, line3, 

    anim = animation.FuncAnimation(fig, animate, frames=len(times2),
                                   interval=1000*max(times2) / len(times2),
                                   blit=True)
    plt.close(fig)
    return anim

anim = animate_pendulum(times2, CPx, CPy, CPz, Pox, Poz)
print('It took {:.3f} sec, before HTML, to run the program'.format(time.time() - start))
HTML(anim.to_jshtml())    # needed, when run on an iPad, I know no other way to do it. It is SLOW!